<a href="https://colab.research.google.com/github/EsteArgen/Aspectos_Aritmeticos_Teoria_Ehrhart/blob/main/Triangulaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

**Aspectos Aritméticos de la Teoría de Ehrhart**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>


<center>

# **Preparación de los archivos**

**🧠Instalación paquetes🫀**

In [1]:

!pip install trimesh plotly mapbox_earcut scipy pyvista

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 711.2/711.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.9 MB/s eta 0:00:00


**🖇️Conectar con Google Drive🔌**

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**🛤️Ruta de los archivos .obj🛣️**

In [3]:

ruta_obj = "/content/drive/Shared drives/POLYTOPES/TRIANGULATIONS/dodecahedron.obj"

**🗂️Módulos para convertir el archivo .obj🗳️**

In [4]:

import os
import trimesh

**😎Verificar existencia del archivo .obj🥸**

In [5]:

print("¿Archivo encontrado?", os.path.exists(ruta_obj))

¿Archivo encontrado? True


**🪫Cargar malla del .obj🔋**

In [6]:

mesh = trimesh.load(ruta_obj)
print(mesh)

<trimesh.Trimesh(vertices.shape=(20, 3), faces.shape=(36, 3))>


# **Procesamiento del poliedro**

**📥Importaciones📦**

In [7]:

import plotly.graph_objects as go
import pyvista as pv
import numpy as np
import trimesh
from trimesh import Trimesh

**👨‍💻Implementación👩‍💻**

In [8]:

def mostrar_malla(mesh, titulo="Triangulación"):
    v = mesh.vertices
    f = mesh.faces
    fig = go.Figure(data=[go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        opacity=0.5, color='lightblue'
    )])
    fig.update_layout(title=titulo, scene=dict(aspectmode='data'))
    fig.show()

**👨‍💻Implementación👩‍💻**

In [9]:

def visualizar_coloreado(mesh, titulo="Malla Triangulada"):
    v = mesh.vertices
    f = mesh.faces

    fig = go.Figure()

    colores = np.random.rand(len(f), 3)

    for i, tri in enumerate(f):
        tri_vertices = v[tri]
        fig.add_trace(go.Mesh3d(
            x=tri_vertices[:, 0],
            y=tri_vertices[:, 1],
            z=tri_vertices[:, 2],
            i=[0], j=[1], k=[2],
            color=f'rgb({colores[i][0]*255:.0f}, {colores[i][1]*255:.0f}, {colores[i][2]*255:.0f})',
            opacity=1.0,
            name=f"Face {i}"
        ))

    fig.update_layout(title=titulo, scene=dict(aspectmode='data'), margin=dict(l=0, r=0, b=0, t=30))
    fig.show()

**👨‍💻Implementación👩‍💻**

In [10]:

def visualizar_transparente_con_bordes(mesh, titulo="Interior visible"):

    v = mesh.vertices
    f = mesh.faces

    fig = go.Figure(data=[go.Mesh3d(
        x=v[:,0], y=v[:,1], z=v[:,2],
        i=f[:,0], j=f[:,1], k=f[:,2],
        color='lightblue', opacity=0.3,
        name="Poliedro"
    )])

    for face in f:
        puntos = v[face]
        edges = [(0,1), (1,2), (2,0)]
        for a,b in edges:
            fig.add_trace(go.Scatter3d(
                x=[puntos[a][0], puntos[b][0]],
                y=[puntos[a][1], puntos[b][1]],
                z=[puntos[a][2], puntos[b][2]],
                mode='lines',
                line=dict(color='black', width=2),
                showlegend=False
            ))

    fig.update_layout(title=titulo, scene=dict(aspectmode='data'), margin=dict(l=0, r=0, b=0, t=30))
    fig.show()


**👨‍💻Implementación👩‍💻**

In [11]:

def tetrahedralizar_y_extraer_superficie(mesh):

    faces_pv = np.hstack([[3, *face] for face in mesh.faces])
    pv_mesh = pv.PolyData(mesh.vertices, faces_pv)

    volumen = pv_mesh.delaunay_3d()

    superficie = volumen.extract_surface()
    v = np.array(superficie.points)
    f = np.array(superficie.faces).reshape((-1, 4))[:, 1:]

    return Trimesh(vertices=v, faces=f)

# **Vista principal**

**👨‍💻Implementación👩‍💻**

In [12]:

visualizar_coloreado(mesh, "Original")

**Vista interior del poliedro**

**👨‍💻Implementación👩‍💻**

In [13]:
visualizar_transparente_con_bordes(mesh, "Interior del Poliedro")

**👨‍💻Implementación👩‍💻**

In [14]:

faces_pv = np.hstack([[3, *face] for face in mesh.faces])
pv_mesh = pv.PolyData(mesh.vertices, faces_pv)
volumen = pv_mesh.delaunay_3d()
superficie = volumen.extract_surface()
v = np.array(superficie.points)
f = np.array(superficie.faces).reshape((-1, 4))[:, 1:]

mesh_tetra = trimesh.Trimesh(vertices=v, faces=f)
visualizar_transparente_con_bordes(mesh_tetra, "Tetrahedralización (volumen interno)")

# **Diferentes triangulaciones**

## **2-Triangulción de las caras**

**👨‍💻Implementación👩‍💻**

In [15]:

triang_2 = mesh.subdivide_to_size(max_edge=0.3)

**Número de triángulos de la triangulación**

**👨‍💻Implementación👩‍💻**

In [16]:

print("Triangulación 2 :", len(triang_2.faces), "triángulos")

Triangulación 2 : 2304 triángulos


**🥸Ver triangulación coloreada🤓**

**👨‍💻Implementación👩‍💻**

In [18]:

visualizar_coloreado(triang_2, "Triangulación 2")